## Anova for our analysis

# Load in packages 

In [29]:
import numpy as np
import mne
from pathlib import Path
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

## Load in the data and epochs

In [30]:
# ALL FROM LAURAS GROUP LEVEL ANALYSIS
# Define the directory
data_path = Path("/Users/lina/Documents/GitHub/ore_EEG/epochs")

# Initialize an empty list to store epochs objects
all_epochs = []

# Iterate over your saved files and load them into epochs objects
for participant in ["own_sub1", "own2_ah"]:
    # Load epochs data from each file
    epochs = mne.read_epochs(data_path / f"epochs_{participant}-epo.fif", verbose=False, preload=True)
    
    # Only keep EEG channels
    epochs.pick_types(eeg=True)
    
    # Append the loaded epochs object to the list
    all_epochs.append(epochs)

# Check the type and length of the list of epochs objects
print(type(all_epochs))  # We have now created a list of epochs objects
print(len(all_epochs))    # We have 2 epochs objects in the list

# Access the first epochs object in the list
print(type(all_epochs[0]))  # We can access the first epochs object in the list which is an Epochs object

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
<class 'list'>
2
<class 'mne.epochs.EpochsFIF'>


## Define channels and time

In [39]:
# chosen channels
picks = ["Oz", "P7"] # just random 

# time window in seconds
tmin = 0.1
tmax = 0.2

## FACTORS for anova

In [40]:
# make lists for labels in conditions
ethnicity_labels = ['white', 'black', 'asian']
emotion_labels = ['happy', 'sad']
data = []

# WE want combinations of ethnicity and emotion, so we loop over all of the epochs conditions
for epochs in all_epochs:
    for ethnicity in ethnicity_labels:
        for emotion in emotion_labels:
                current_epochs = epochs[f'{emotion}/{ethnicity}']
                # get data
                data_current = current_epochs.get_data(picks=picks, tmin=tmin, tmax=tmax)
                # calculate mean for this data
                data_mean = np.mean(data_current, axis=(0, 1, 2))
                # add data to list
                data.append([ethnicity, emotion, data_mean])

# Make to dtaframe
df = pd.DataFrame(data, columns=['Ethnicity', 'Emotion', 'Mean'])
# see how it looks
print(df)


   Ethnicity Emotion          Mean
0      white   happy -1.427985e-08
1      white     sad -1.886132e-08
2      black   happy -3.755538e-08
3      black     sad  1.858291e-08
4      asian   happy  6.384286e-08
5      asian     sad -1.005507e-07
6      white   happy -2.250763e-07
7      white     sad -4.321362e-07
8      black   happy -3.916005e-07
9      black     sad -3.841368e-07
10     asian   happy -2.706392e-07
11     asian     sad -2.791774e-07


In [45]:
# RUN THE ANOVA
# Get ethinities and emotions conditions
ethnicities = df['Ethnicity'].unique()
emotions = df['Emotion'].unique()

# empty list to store results
anova_results = []

# loop through each combination of ethnicity and emotion
for ethnicity in ethnicities:
    for emotion in emotions:
        # select the data corresponding to current one
        data_subset = df[(df['Ethnicity'] == ethnicity) & (df['Emotion'] == emotion)]['Mean'].values
        
        # we append the data subset to the list
        anova_results.append(data_subset)

# TWO WAY ANOVA
# we fit the model
model = ols('Mean ~ C(Ethnicity) + C(Emotion) + C(Ethnicity):C(Emotion)', data=df).fit()

# table of results
anova_table = sm.stats.anova_lm(model)

# PRINTTTTT
print(anova_table)


                          df        sum_sq       mean_sq         F    PR(>F)
C(Ethnicity)             2.0  5.417652e-15  2.708826e-15  0.050276  0.951363
C(Emotion)               1.0  8.585208e-15  8.585208e-15  0.159342  0.703583
C(Ethnicity):C(Emotion)  2.0  1.110046e-14  5.550232e-15  0.103013  0.903677
Residual                 6.0  3.232741e-13  5.387902e-14       NaN       NaN


# The same but using the neutral condition and leaving out the asians

## Factors

In [49]:
# make lists for labels in conditions
ethnicity_labels = ['white', 'black']
emotion_labels = ['happy', 'neutral', 'sad']
data = []

# WE want combinations of ethnicity and emotion, so we loop over all of the epochs conditions
for epochs in all_epochs:
    for ethnicity in ethnicity_labels:
        for emotion in emotion_labels:
                current_epochs = epochs[f'{emotion}/{ethnicity}']
                # get data
                data_current = current_epochs.get_data(picks=picks, tmin=tmin, tmax=tmax)
                # calculate mean for this data
                data_mean = np.mean(data_current, axis=(0, 1, 2))
                # add data to list
                data.append([ethnicity, emotion, data_mean])

# Make to dtaframe
df = pd.DataFrame(data, columns=['Ethnicity', 'Emotion', 'Mean'])
# see hot it looks
print(df)


   Ethnicity  Emotion          Mean
0      white    happy -1.427985e-08
1      white  neutral -7.863522e-07
2      white      sad -1.886132e-08
3      black    happy -3.755538e-08
4      black  neutral -2.942374e-07
5      black      sad  1.858291e-08
6      white    happy -2.250763e-07
7      white  neutral -3.468075e-07
8      white      sad -4.321362e-07
9      black    happy -3.916005e-07
10     black  neutral -4.915243e-07
11     black      sad -3.841368e-07


In [50]:
# copy from above :DDDDD
# Get ethinities and emotions conditions
ethnicities = df['Ethnicity'].unique()
emotions = df['Emotion'].unique()

# empty list to store results
anova_results = []

# loop through each combination of ethnicity and emotion
for ethnicity in ethnicities:
    for emotion in emotions:
        # select the data corresponding to current one
        data_subset = df[(df['Ethnicity'] == ethnicity) & (df['Emotion'] == emotion)]['Mean'].values
        
        # we append the data subset to the list
        anova_results.append(data_subset)

# TWO WAY ANOVA
# we fit the model
model = ols('Mean ~ C(Ethnicity) + C(Emotion) + C(Ethnicity):C(Emotion)', data=df).fit()

# table of results
anova_table = sm.stats.anova_lm(model)

# PRINTTTTT
print(anova_table)


                          df        sum_sq       mean_sq         F    PR(>F)
C(Ethnicity)             1.0  4.922443e-15  4.922443e-15  0.080379  0.786313
C(Emotion)               2.0  2.333882e-13  1.166941e-13  1.905510  0.228724
C(Ethnicity):C(Emotion)  2.0  3.608003e-14  1.804001e-14  0.294577  0.755031
Residual                 6.0  3.674420e-13  6.124034e-14       NaN       NaN
